In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

C:\Users\ravindra\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
gatrain = pd.read_csv('gender_age_train.csv', index_col='device_id')
#gatest = pd.read_csv('gender_age_test.csv', index_col = 'device_id')
phone = pd.read_csv('phone_brand_device_model.csv')
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv('events.csv', parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv('app_events.csv', usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv('app_labels.csv')

In [41]:
# quick check to find duplicate values
# probably we need to see if there is duplicated values, that hurt our model
# 529 devices have duplicate values = give a device id, it can belong to only one brand
len(pd.read_csv('phone_brand_device_model.csv').set_index('device_id').index.get_duplicates())

529

In [42]:
# just select first 5 rows
# build your features for just 5 row
# because running query on full training set took 6+ hours and still unfinished!!!
gatrain = gatrain.head()
gatrain

,gender,age,group
device_id,,,
-8076087639492063270,M,35,M32-38
-2897161552818060146,M,35,M32-38
-8260683887967679142,M,35,M32-38
-4938849341048082022,M,30,M29-31
245133531816851882,M,30,M29-31


We are going to build Compressed Sparse row matrix, so build row index for train and test. <br/>
Read about it here,
http://www.scipy-lectures.org/advanced/scipy_sparse/csr_matrix.html  <br/>
Basically we have lots of feature possible, nxm matrix has fewer ones and lots of zeros. Save space and do update faster by compressed sparse matrix which can be row or column.


In [43]:
gatrain['trainrow'] = np.arange(gatrain.shape[0])
gatrain.head()

,gender,age,group,trainrow
device_id,,,,
-8076087639492063270,M,35,M32-38,0
-2897161552818060146,M,35,M32-38,1
-8260683887967679142,M,35,M32-38,2
-4938849341048082022,M,30,M29-31,3
245133531816851882,M,30,M29-31,4


In [44]:
# we will use this list for filtering and reducing dataframe
device_id_list = list(gatrain.index)
print device_id_list

[-8076087639492063270, -2897161552818060146, -8260683887967679142, -4938849341048082022, 245133531816851882]


In [45]:
# now reduce events table
events = events[events['device_id'].isin(device_id_list)]
# so we have some device id for which events are missing !!!
# may be we need to model them using separate models ??
events.head()

,device_id,timestamp,longitude,latitude
event_id,,,,
2479656,-8260683887967679142,2016-05-01 14:23:37,0.0,0.0


In [46]:
# we might want to do one hot encoding, label encoder enforces ordering which might be bad choice
# read more about it here http://datascience.stackexchange.com/questions/9443/when-to-use-one-hot-encoding-vs-labelencoder-vs-dictvectorizor
# No, I was wrong. Label encoding is transformed to one hot encoding via CSR. see below.
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
# since index_col set as "device_id" so directly assigning
# gatrain['brand'] does find/match and update simultaneously.
# I felt this is really cool trik :)
gatrain['brand'] = phone['brand']
#gatest['brand'] = phone['brand']

In [47]:
gatrain.head()

,gender,age,group,trainrow,brand
device_id,,,,,
-8076087639492063270,M,35,M32-38,0,51
-2897161552818060146,M,35,M32-38,1,51
-8260683887967679142,M,35,M32-38,2,51
-4938849341048082022,M,30,M29-31,3,51
245133531816851882,M,30,M29-31,4,51


In [48]:
# basically here we are building huge matrix where each row has single '1'.
# 131 features -> out of which only one brand is 1 all others are 0.
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
#Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
#                       (gatest.testrow, gatest.brand)))
print('Brand features: train shape {}'.format(Xtr_brand.shape))#, Xte_brand.shape))

Brand features: train shape (5, 52)


In [49]:
# now do the same drill for phone brand + device_model
# we concatenate the strings of phone brand and device model
m = phone.phone_brand.str.cat(phone.device_model)
m[0:5]

device_id
-8890648629457979026               小米红米
 1277779817574759137             小米MI 2
 5137427614288105724        三星Galaxy S4
 3669464369358936369          SUGAR时尚手机
-5019277647504317457    三星Galaxy Note 2
Name: phone_brand, dtype: object

In [50]:
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
#gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
#Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
#                       (gatest.testrow, gatest.model)))
print('Model features: train shape {}'.format(Xtr_model.shape))#, Xte_model.shape))

Model features: train shape (5, 866)


In [51]:
appevents.head()

,event_id,app_id,is_active
0,2,5927333115845830913,True
1,2,-5720078949152207372,False
2,2,-1633887856876571208,False
3,2,-653184325010919369,True
4,2,8693964245073640147,True


In [52]:
appencoder = LabelEncoder().fit(appevents.app_id)
appevents['app'] = appencoder.transform(appevents.app_id)
print len(appencoder.classes_)
appevents.head()

19237


,event_id,app_id,is_active,app
0,2,5927333115845830913,True,15408
1,2,-5720078949152207372,False,3384
2,2,-1633887856876571208,False,7620
3,2,-653184325010919369,True,8902
4,2,8693964245073640147,True,18686


In [53]:
events.head()

,device_id,timestamp,longitude,latitude
event_id,,,,
2479656,-8260683887967679142,2016-05-01 14:23:37,0.0,0.0


In [54]:
# Merge method	SQL Join Name	Description
# left	LEFT OUTER JOIN	Use keys from left frame only
# right	RIGHT OUTER JOIN	Use keys from right frame only
# outer	FULL OUTER JOIN	Use union of keys from both frames
# inner	INNER JOIN	Use intersection of keys from both frames
#
# you want to merge device_id from events table using left join based on event_id
# use right_index which is event_id for matching(join) key.
# I have changed it to right join as we are interested in collecting events for one device id
app_events_device_id = appevents.merge(events[['device_id']], how='right', left_on='event_id', right_index=True)
app_events_device_id.head()

,event_id,app_id,is_active,app,device_id
24761504,2479656,4287147352639325907,False,13855,-8260683887967679142
24761505,2479656,-4085686983594738208,False,5019,-8260683887967679142
24761506,2479656,6324194957022958681,False,15873,-8260683887967679142
24761507,2479656,-8687507491028761317,False,526,-8260683887967679142
24761508,2479656,877288515501232570,True,10648,-8260683887967679142


In [55]:
# apps events per device
apps_events_per_device = app_events_device_id.groupby(['device_id', 'app'])
for key, row in apps_events_per_device:
    print key
    print row
    break

(-8260683887967679142, 135)
          event_id               app_id is_active  app            device_id
24761547   2479656 -9104163784380174291     False  135 -8260683887967679142


In [35]:
# literally froze my computer when ran with full set
# def extract_app_events_features(group):
#    a = group['app'].agg('size')
#    b = group['is_active'].agg('sum')
#    return a, b
# queires for number of types app events happened
(app_events_device_id.groupby(['device_id', 'app'])['app'].agg('size')
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
#apps_events_per_device['app'].agg('size').merge(apps_events_per_device['is_active'].agg('sum'), how='left', left_index=True, right_index=True)
# I am also adding number of times app was active
# apps_events_per_device['is_active'].agg('sum')
# dont run below code will freeze
# apps_events_per_device.apply(extract_app_events_features)

AttributeError: 'Series' object has no attribute 'merge'

In [58]:
# app -> label encoded value of original app id
# size -> number of events collected for that id
# we could add number of times particular app was active.
# usage stastics based on weekly, hour or monthly basis.
deviceapps = (appevents.merge(events[['device_id']], how='right',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
deviceapps.head()

,device_id,app,size,trainrow
0,-8260683887967679142,135,1,2
1,-8260683887967679142,267,1,2
2,-8260683887967679142,317,1,2
3,-8260683887967679142,526,1,2
4,-8260683887967679142,974,1,2


In [59]:
# moral of the story.
# totally 53 apps were installed on the device
len(deviceapps)

53

In [62]:
# now build CSR matrix same as earlier.
# actually no need to do dropna for this script, but for full set its needed
napps = len(appencoder.classes_)
d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(gatrain.shape[0],napps))
print('Apps data: train shape {}'.format(Xtr_app.shape))

Apps data: train shape (5, 19237)


In [64]:
Xtrain = hstack((Xtr_brand, Xtr_model, Xtr_app), format='csr')
# simple feature engineering set results in 20k feature.
print('All features: train shape {}'.format(Xtrain.shape))

All features: train shape (5, 20155)
